<a href="https://colab.research.google.com/github/CallMeL/IDS-for-In-Vehicle-Network/blob/main/GIDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# overview
对于论文[《GIDS: GAN based Intrusion Detection System for In-Vehicle Network》](https://awesong-kor.github.io/files/GIDS%20GAN%20based%20Intrusion%20Detection%20System%20for%20In-Vehicle%20Network.pdf)的代码调试


# 参考
[文献笔记 —— GIDS: GAN based Intrusion Detection System for In-Vehicle Network](https://blog.csdn.net/weixin_45620107/article/details/110933886)

[代码：created one-hot-vectors and batch of 64 CAN Images](https://github.com/NikosMouzakitis/GAN_based_Intrusion_Detection_System_for_in-vehicle_network/blob/main/gan_based_intrusion_detection.py)

[CAN2.0B 数据帧详解](https://blog.csdn.net/daweihuahuo/article/details/78352762)


# 论文阅读


## 概述
车辆中的控制器局域网(CAN)总线是一种高效的标准总线，支持所有电子控制单元(ECU)之间的通信。但是，由于缺乏安全功能，CAN总线不足以保护自身。

为了有效地检测可疑网络连接，需要**入侵检测系统(IDS)**。 

与用于Internet的传统IDS不同，用于车辆网络的已知攻击特征很少。此外，用于车辆的IDS要求很高的准确性，因为任何假阳性错误都会严重影响驾驶员的安全。 

为了解决这个问题，论文提出一种新的车载网络IDS模型GIDS(基于GAN的入侵检测系统)，它使用了深度学习模型Generative Adversarial Nets。 **GIDS可以学会仅使用正常数据来检测未知攻击。**

## 要点


*   从CAN数据中提取了CAN ID的模式进行训练
  * CAN数据包含信息太多了、全部提取浪费时间
  * 通过使用one-hot向量编码将提取的CAN ID转换为简单图像：将CAN ID的每个元素的二进制形式编码为one-hot向量（eg: canID =  0x2a0, 它由3个one-hot向量组成，例如[0100 ... 000]，[0..0100000]和[0..1000000]。2在第二位为1，a在第十位为1，以此类推

## 整体设计

1)针对已知攻击的训练：第一**鉴别器**接收从实际车辆提取的正常CAN图像和异常CAN图像。因为第一个鉴别器在训练过程中使用攻击数据，所以可以检测到的攻击类型可能仅限于训练中使用的攻击。

2)训练未知攻击：通过对抗过程同时训练**生成器G**和第二个**鉴别器**。生成器通过使用随机噪声生成伪图像。第二个鉴别器接收正常的CAN图像和生成器生成的伪图像，并估计接收到的图像为真实CAN图像的可能性。即，第二鉴别器鉴别输入图像是真实的CAN图像还是由生成器生成的伪图像。生成器和第二个鉴别器相互竞争，并提高了性能。在GIDS模型中，第二个鉴别器最终赢得了生成器，因此第二个鉴别器甚至可以检测到类似于真实CAN图像的伪图像


# 代码

## 数据集

## 网络

### OHV(one-hot-vector)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
#import tensorflow as tf

def one_hot_vector(a):
	'''create a OHV(one-hot-vector) and return it based on 
	the hexadicimal digit string as provided argument.'''

	ret= np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
	if(a=='0'):
		ret[0]=1
	elif(a=='1'):	
		ret[1]=1
	elif(a=='2'):	
		ret[2]=1
	elif(a=='3'):	
		ret[3]=1
	elif(a=='4'):	
		ret[4]=1
	elif(a=='5'):	
		ret[5]=1
	elif(a=='6'):	
		ret[6]=1
	elif(a=='7'):	
		ret[7]=1
	elif(a=='8'):	
		ret[8]=1
	elif(a=='9'):	
		ret[9]=1
	elif(a=='a'):
		ret[10]=1
	elif(a=='b'):	
		ret[11]=1
	elif(a=='c'):	
		ret[12]=1
	elif(a=='d'):	
		ret[13]=1
	elif(a=='e'):	
		ret[14]=1
	elif(a=='f'):	
		ret[15]=1

	return ret

def create_CAN_image(s):
	'''Function which is creating the CAN Image of a given ID, by
	   subsequent calls to one_hot_vector() and returns as a numpy 
	   array the CAN Image created.'''
	
	a = one_hot_vector(s[0])
	b = one_hot_vector(s[1])
	c = one_hot_vector(s[2])

	ret=np.array([a,b,c])

	return ret

def create_64batch_Discriminator(can_image):
    '''function that creates a 64X48 batch of
    the CAN images.'''
    
    cnt=0
    for i in can_image:
        j=i.reshape(1,48)
       
        if(cnt==0):
            a=j
            cnt+=1
        else:
            a=np.concatenate((a,j),axis=0)
    return a
    
    
#start of the program
    

print("GAN Intrusion Detection System for CAN Bus case study.\nStarted.")
ids_normal_list=[]
f=open("NORMAL_IDS.txt","r")
lines=f.readlines()
f.close()

for i in lines:
	i=i[2:len(i)-1]	
	ids_normal_list.append(i)

ids_normal_list.pop()
#now we have stored in ids_list all the IDS of the normal run.

#create all the normal CAN_IMAGES(3*16 arrays)
normal_CAN_Images=[]

for i in ids_normal_list:
	normal_CAN_Images.append(create_CAN_image(i))

#create a batch of 64X16*3 2d image
#batchDiscriminator = create_64batch_Discriminator(normal_CAN_Images[0:64])
#plt.figure(1)
#plt.imshow(batchDiscriminator,cmap="binary")
#plt.title("A batch for the Discriminator D(64X48)")

normal_train_batchD = [] #training list for batches of normal
count_normal_train_batchD = 0

for i in range(0,10816,64):
    normal_train_batchD.append(create_64batch_Discriminator(normal_CAN_Images[i:i+64]))
    count_normal_train_batchD += 1
    

### GAN